In [12]:
#svm
import numpy as np
from sklearn import svm
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

In [17]:
#def linear_kernel(x,y):
#    return np.dot(x,y.T)

def polynomial_kernel(X1,X2,d=2):
    gram_matrix = np.zeros((X1.shape[0], X2.shape[0]))
    for i, x1 in enumerate(X1):
        for j, x2 in enumerate(X2):
            x1 = x1.flatten()
            x2 = x2.flatten()
            gram_matrix[i, j] = pow((np.dot(x1,x2)+1),d)
    return gram_matrix


def gaussianKernelGramMatrix(X1, X2, sigma=0.1):
    gram_matrix = np.zeros((X1.shape[0], X2.shape[0]))
    for i, x1 in enumerate(X1):
        for j, x2 in enumerate(X2):
            x1 = x1.flatten()
            x2 = x2.flatten()
            gram_matrix[i, j] = np.exp(- np.sum( np.power((x1 - x2),2) ) / float( 2*(sigma**2) ) )
    return gram_matrix

class multikernel:
    

In [18]:
column_names = ['Age','workclass','fnlgwt','edu','education_num','marital_status','occ','rel_status','race',
               'sex','capital_gain','capital_loss','hrs_p_week','native_Country','label']
column_names_test = ['Age','workclass','fnlgwt','edu','education_num','marital_status','occ','rel_status','race',
               'sex','capital_gain','capital_loss','hrs_p_week','native_Country']
train_data = pd.read_csv("C:\\Users\\HP\\Documents\\IIT HYDERABAD NOTES\\Applied ML\\svm\\data.tar\\data\\train.csv",
                        names = column_names)
test_data  = pd.read_csv("C:\\Users\\HP\\Documents\\IIT HYDERABAD NOTES\\Applied ML\\svm\\data.tar\\data\\test.csv",
                        names = column_names_test)

train_data['Age'].fillna((train_data['Age'].mean()), inplace=True)
train_data['fnlgwt'].fillna((train_data['fnlgwt'].mean()), inplace=True)
train_data['education_num'].fillna((train_data['education_num'].mean()), inplace=True)
train_data['capital_gain'].fillna((train_data['capital_gain'].mean()), inplace=True)
train_data['capital_loss'].fillna((train_data['capital_loss'].mean()), inplace=True)
train_data['hrs_p_week'].fillna((train_data['hrs_p_week'].mean()), inplace=True)

test_data['Age'].fillna((train_data['Age'].mean()), inplace=True)
test_data['fnlgwt'].fillna((train_data['fnlgwt'].mean()), inplace=True)
test_data['education_num'].fillna((train_data['education_num'].mean()), inplace=True)
test_data['capital_gain'].fillna((train_data['capital_gain'].mean()), inplace=True)
test_data['capital_loss'].fillna((train_data['capital_loss'].mean()), inplace=True)
test_data['hrs_p_week'].fillna((train_data['hrs_p_week'].mean()), inplace=True)
#print(train_data['label'])

In [19]:
train_data['workclass'] = train_data['workclass'].astype('category')
train_data['edu'] = train_data['edu'].astype('category')
train_data['marital_status'] = train_data['marital_status'].astype('category')
train_data['occ'] = train_data['occ'].astype('category')
train_data['rel_status'] = train_data['rel_status'].astype('category')
train_data['race'] = train_data['race'].astype('category')
train_data['sex'] = train_data['workclass'].astype('category')
train_data['native_Country'] = train_data['workclass'].astype('category')

test_data['workclass'] = train_data['workclass'].astype('category')
test_data['edu'] = train_data['edu'].astype('category')
test_data['marital_status'] = train_data['marital_status'].astype('category')
test_data['occ'] = train_data['occ'].astype('category')
test_data['rel_status'] = train_data['rel_status'].astype('category')
test_data['race'] = train_data['race'].astype('category')
test_data['sex'] = train_data['workclass'].astype('category')
test_data['native_Country'] = train_data['workclass'].astype('category')


In [25]:
cat_cols = train_data.select_dtypes(['category']).columns
#print(cat_cols)
train_data[cat_cols] = train_data[cat_cols].apply(lambda x : x.cat.codes)
test_data[cat_cols] = test_data[cat_cols].apply(lambda x : x.cat.codes)

split_criteria = np.random.rand(len(train_data))<1
train_data_cut = train_data[split_criteria]


#data_set = train_data.values.tolist()
labels = np.array(train_data_cut['label'])
#print(train_data.head())
train_data_cut = train_data_cut.drop(["label"],axis=1)
data_set = np.array(train_data_cut)


classif_linear= svm.SVC(kernel = "precomputed")
classif_polynomial = svm.SVC(kernel = "precomputed")
classif_gaussian = svm.SVC(kernel ="precomputed")

accuracy_linear = 0
accuracy_gaussian = 0
accuracy_poly = 0

kfold = StratifiedKFold(n_splits=5,shuffle=True)
for train_indexes, test_indexes in kfold.split(data_set,labels):
    
    trainX, trainY = data_set[train_indexes], labels[train_indexes]
    testX, testY = data_set[test_indexes], labels[test_indexes]
    
    linear_gram_train = np.dot(trainX,trainX.T)
    linear_gram_test = np.dot(testX,trainX.T)
    
    gaussian_gram_train = gaussianKernelGramMatrix(trainX,trainX)
    gaussian_gram_test = gaussianKernelGramMatrix(testX,trainX)
    
    polynomial_gram_train = polynomial_kernel(trainX,trainX,3)
    polynomial_gram_test = polynomial_kernel(testX,trainX,3)
    
    classif_linear.fit(linear_gram_train,trainY)
    classif_gaussian.fit(gaussian_gram_train,trainY)
    classif_polynomial.fit(polynomial_gram_train,trainY)
    
    pred_linear = classif_linear.predict(linear_gram_test)
    pred_gaussian = classif_gaussian.predict(gaussian_gram_test)
    pred_polynomial = classif_polynomial.predict(polynomial_gram_test)
    
    accuracy_linear += accuracy_score(testY,pred_linear)#classif_linear.score(testX,testY)
    accuracy_poly += accuracy_score(testY,pred_polynomial)
    accuracy_gaussian += accuracy_score(testY, pred_gaussian)#classif_gaussian.score(testX,testY)

print("linear kernel accuracy")
print(float(accuracy_linear)/5)
print("gaussian kernel accuracy")
print(float(accuracy_gaussian)/5)
print("polynomia kernel accuracy")
print(float(accuracy_poly)/5)

linear kernel accuracy
0.8054089801601115
gaussian kernel accuracy
0.7759415245388096
polynomia kernel accuracy
0.6733379742429516
